In [ ]:
from py2neo import Graph, Node, Relationship
import csv
import os

In [ ]:
host = 'bolt://localhost:7687'
auth = ('neo4j', 'ibdpract2')

graph = Graph(host, auth=auth)

In [ ]:
directory = os.path.join(os.getcwd(), "csvs")

csv_file = os.path.join(directory, "general.csv")

with open(csv_file, "r") as file:
    reader = csv.DictReader(file)

    for row in reader:
        paper_id = row['paperId']
        title = row['title']

        author_ids = row['authorId'].split(',')
        author_names = row['authorName'].split(',')

        # Crear nodo de papel (Paper)
        paper = Node('Paper',
                     paperId=paper_id,
                     title=title)
        graph.create(paper)

        # Crear nodos de autores (Author) y relaciones (written) con el papel (Paper)
        for author_id, author_name in zip(author_ids, author_names):
            author = Node('Author', authorId=author_id, name=author_name)
            graph.create(author)

            relationship = Relationship(author, "written", paper)
            graph.create(relationship)

In [ ]:
author_name = input("Ingrese el nombre del autor: ")

query = """
MATCH (author:Author {name: $author_name})-[:written]->(paper:Paper)<-[:written]-(coauthor:Author)
RETURN DISTINCT paper.title, count(DISTINCT coauthor) AS num_authors
ORDER BY num_authors ASC
"""

result = graph.run(query, author_name=author_name)

for record in result:
    print(record["paper.title"], record["num_authors"])